# INIT

## load pack

In [1]:
import sys
sys.path.append("../POS_tagging/")
sys.path.append("../../../pomegranate/")
sys.path.append(r"../../../WhoCell\pomegranate")
sys.path.append(r"../../../WhoCell")
sys.path.append(r"../../../WhoCell/")
sys.path.append(r"../../../WhoCell/who_cell/")
sys.path.append(r"../../../WhoCell/who_cell/models/")
sys.path.append(r"../../../WhoCell/who_cell/simulation/")
sys.path.append(r"../../../WhoCell/who_cell/experiments/")

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import string

from collections import Counter
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
import pandas as pd
import seaborn as sns
import itertools
from functools import reduce
import pickle

from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

from simulation_for_gibbs import Simulator_for_Gibbs
from gibbs_sampler import GibbsSampler
from experiment_report import ExperimentReport 
from gibbs_experiments import GibbsExperiment
from pos_data_builder import PosDataBuilder
from pos_code_base import PosCodeBase
from em_sequence_labeling import EmSequenceLabeling


In [3]:
%connect_info
%load_ext autoreload

import warnings
warnings.filterwarnings('ignore')

{
  "shell_port": 55059,
  "iopub_port": 39709,
  "stdin_port": 47695,
  "control_port": 49539,
  "hb_port": 44215,
  "ip": "127.0.0.1",
  "key": "fd44b18a-9ba6e7b4c71af682f5025dd7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-7c5fbb8d-899e-49e7-b077-da8912056e86.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


## load experiments sets

In [4]:
data_builder = PosDataBuilder()
code_base = PosCodeBase()

# Build experiment 

The experiment details -   

Our data :   
1)Training data : labeled sentences
2)Test set - the same

Experiment :  
1)We will use the training set labels for few observations Markov chain - not hidden! by using the states labels as emissions.  
2)Then we learn the emissions from the training set  
3)We will predict the labels on the test set   



In [5]:
pc = 0.5

test_set_words, test_set_tags,train_set_words, train_set_tags = data_builder.get_experiment_sets_from_real_data(pc,False)
all_states = data_builder._build_emissions_probabilites().keys()

In [11]:
data_builder.

TypeError: object of type 'NoneType' has no len()

In [6]:
transitions_probs,transitions_probs_df = data_builder.get_known_transitions()

In [7]:
number_of_iters = 20

## Learning without "few observations" 

In [8]:
emm_probs = {state:{state:1} for state in all_states}


In [9]:
first_experiment_args = {"test_set_words":train_set_tags,
"start_probs":data_builder._build_starting_probabilites(),
"emms_probs":emm_probs,
"number_of_iters":number_of_iters,
"N":2,
"is_only_observed":"observed",
"description":"P(C) = 1",
"known_transitions":None,
"comper_transitions":None,
"comper_transitions_title" :None,
"state_order_for_plot":data_builder.STATE_ORDER_TO_PLOT,
"title":"Naive MM"}
first_experiment_results = code_base.run_pos_experiment(**first_experiment_args)

 10%|████▍                                       | 2/20 [00:53<08:05, 26.99s/it]


KeyboardInterrupt: 

## learning when the expected trajectory is twice the length

In [ ]:
_known_N = [2*len(seq) for seq in train_set_tags]

In [ ]:
# second_experiment_args = {"test_set_words":train_set_tags,
# "start_probs":data_builder._build_starting_probabilites(),
# "emms_probs":emm_probs,
# "number_of_iters":number_of_iters,
# "N":_known_N,
# "is_only_observed":"all",
# "description":"P(C) = 1",
# "known_transitions":None,
# "comper_transitions":None,
# "comper_transitions_title" :None,
# "state_order_for_plot":data_builder.STATE_ORDER_TO_PLOT,
# "title":"few obs"}
# second_experiment_results = code_base.run_pos_experiment(**second_experiment_args)

## learning when the expected trajectory is twice the length only observed

In [ ]:
third_experiment_args = {"test_set_words":train_set_tags,
"start_probs":data_builder._build_starting_probabilites(),
"emms_probs":emm_probs,
"number_of_iters":number_of_iters,
"N":_known_N,
"is_only_observed":"extended",
"description":"P(C) = 1",
"known_transitions":None,
"comper_transitions":None,
"comper_transitions_title" :None,
"state_order_for_plot":data_builder.STATE_ORDER_TO_PLOT,
"title":"few obs only seen"}
third_experiment_results = code_base.run_pos_experiment(**third_experiment_args)

# measures and plots

In [ ]:
def build_pome_model_from_trnaisiotns(predicted_transitions,words_emms_probs,start_probs,all_states):
    states_name_to_state_mapping = {state:State(DiscreteDistribution(words_emms_probs[state]),state) for state in all_states}

    _model = HiddenMarkovModel()
    for _from,_tos in predicted_transitions.items() : 
        if _from == "start" : 
            for state in all_states : 
                _to_state = states_name_to_state_mapping[state]
                _model.add_transition(_model.start,_to_state,start_probs[state])
            continue
        for _to,val in _tos.items() :
            if _to == 'end' : continue
            _to_state = states_name_to_state_mapping[_to]
            
            _from_state = states_name_to_state_mapping[_from]
            _model.add_transition(_from_state,_to_state,val)

    _model.bake()
    return _model

def calculate_error_pome(predicted_transitions,test_set_words,test_set_tags,words_emms_probs,start_probs,all_states,unknown_words):
    pome_model = build_pome_model_from_trnaisiotns(predicted_transitions,words_emms_probs,start_probs,all_states)
    states_name_list = [state.name for state in pome_model.states]

    errors = []
    for sent_words,sent_tags in zip(test_set_words,test_set_tags) :
        sent_words = [(word if word not in unknown_words else None) for word in sent_words]
        _predicted = pome_model.predict(sent_words)
        predicted_tags = [states_name_list[i] for i in _predicted]
        error = sum([i!=j for i,j in zip(sent_tags,predicted_tags)])
        errors.append(error)
    
    amount_of_tags = sum(list(map(len,test_set_tags)))
    return np.sum(errors)/amount_of_tags

def calculate_error_gibbs(N,predicted_transitions,test_set_words,
                          test_set_tags,words_emms_probs,start_probs,
                          all_states):    
    
    _test_set_words = list(filter(lambda x:len(x)<60,test_set_words))
    _test_set_tags = list(filter(lambda x:len(x)<60,test_set_tags))
    gibbs_sampler = GibbsSampler(N,multi_process=True)
    all_states_picked_by_w = EmSequenceLabeling.sequence_labeling_known_emissions(_test_set_words,
                                                                                   predicted_transitions,
                                                                                   start_probs,
                               words_emms_probs, 5, N = N)
    
    errors = []
    for known_tags,predicted_tags in zip(_test_set_tags,all_states_picked_by_w[-1]) :
        error = sum([i!=j for i,j in zip(known_tags,predicted_tags)])
        errors.append(error)
    
    amount_of_tags = sum(list(map(len,_test_set_tags)))
    return np.sum(errors)/amount_of_tags


In [ ]:
def plot_df_results(model_results_df,title):
    
    fig, sub = plt.subplots(1,1,figsize=(8, 8))
    

    sns.lineplot(data=model_results_df, ax=sub, legend='full', dashes=False)
    sub.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    sub.set_title(title)
    sub.set_xlabel("iter")
    sub.set_ylabel(f"Hamming distance over test set")

    plt.subplots_adjust(hspace=0.8)
    plt.subplots_adjust(wspace=0.8)

    plt.show()

def build_and_plot_results(experiments_results,experiments_params,test_set_words,test_set_tags,data_builder,all_states): 
    words_emms_probs = data_builder._build_emissions_probabilites()

    all_words_in_test_set = data_builder.row_data.vocab
    all_known_words = frozenset(chain(*[val for val in words_emms_probs.values()]))
    unknown_words = (all_words_in_test_set - all_known_words)
    
    words_emms_probs = data_builder._build_emissions_probabilites()
    start_probs = data_builder._build_starting_probabilites()
    
    sentences_length = list(map(len,test_set_tags))
    
#     with open("build_and_plot_data.pkl",'wb') as f : 
#         pickle.dump((experiments_results,experiments_params,test_set_words,test_set_tags,list(all_states),words_emms_probs,all_words_in_test_set,words_emms_probs,start_probs),f)

    
    pomegranate_errors = {}
    missing_obs_errors = {}
    gibbs_no_missing_errors = {}
    
    n_iter_exp = len(experiments_results[0]['transitions'])
    transitions_to_pick = sorted(list(set(list(range(0,n_iter_exp,15)) + [n_iter_exp-1])))
    print(transitions_to_pick)
    for i,(exp_res,exp_args) in enumerate(zip(experiments_results,experiments_params)) :
        transitions = exp_res['transitions']
        fewer_transitions = [transitions[i] for i in transitions_to_pick]
        
        title = exp_args['title']
#         pomegranate_error_per_experiment = [calculate_error_pome(_trans,
#                                                                  test_set_words,test_set_tags,
#                                                                  words_emms_probs,
#                                                                  start_probs,
#                                                                  all_states,
#                                                                  unknown_words) for _trans in fewer_transitions]
        
#         pomegranate_errors[title] = pomegranate_error_per_experiment
        
        gibbs_missing_error_per_experiment = [calculate_error_gibbs(exp_args["N"],
                                                                    _trans,
                                                                    test_set_words,test_set_tags,
                                                                    words_emms_probs,start_probs,
                                                                    all_states) for _trans in fewer_transitions]
        missing_obs_errors[title] = gibbs_missing_error_per_experiment
        
        gibbs_full_error_per_experiment = [calculate_error_gibbs(2,_trans,test_set_words,test_set_tags,
                                                                    words_emms_probs,start_probs,all_states) 
                                           for _trans in fewer_transitions]
        gibbs_no_missing_errors[title] = gibbs_full_error_per_experiment
        
        
#     pome_results_df = pd.DataFrame(pomegranate_errors)
    gibbs_mising_results_df = pd.DataFrame(missing_obs_errors)
    gibbs_full_results_df = pd.DataFrame(gibbs_no_missing_errors)
    
#     plot_df_results(pome_results_df,"inference pomegranate using viterbi")
    plot_df_results(gibbs_mising_results_df,"inference Gibbs counting for missing observations ")
    plot_df_results(gibbs_full_results_df,"inference Gibbs not counting for missing observations")
    
    
        
        
    
    
        
        
    
    

In [ ]:
experiments_results = [first_experiment_results,third_experiment_results]
experiments_params = [first_experiment_args,third_experiment_args]

build_and_plot_results(experiments_results,experiments_params,
                       test_set_words,test_set_tags,data_builder,
                       all_states)


In [ ]:
experiments_results[0]['transitions'][-1]

In [ ]:
experiments_results = [first_experiment_results,second_experiment_results]
experiments_params = [first_experiment_args,second_experiment_args]

build_and_plot_results(experiments_results,experiments_params,
                       test_set_words,test_set_tags,data_builder,
                       all_states)


In [ ]:
n_iter_exp = 100
